# DiGraph - Parallel Flow

We now create a slightly more complex flow:

1. A writer drafts a paragraph.
2. Two editors independently edit for grammar and style (parallel fan-out).
3. A final reviewer consolidates their edits (join).

##### Make sure to update autogen AgentChat to latest version and also remove autogenstudio if it causes any depdendency issues

In [19]:
import asyncio
from autogen_agentchat.agents import AssistantAgent,UserProxyAgent
from autogen_agentchat.teams import SelectorGroupChat
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.conditions import TextMentionTermination
from dotenv import load_dotenv
from autogen_agentchat.ui import Console
import os

# Load API key
load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

# Model client
model_client = OpenAIChatCompletionClient(model='gpt-4o', api_key=api_key)

In [27]:
writer = AssistantAgent(
    name="Writer",
    description="A writer agent that generates text based on user input.",
    model_client=model_client,
    # system_message="You are a creative writer. Please write a story based on Climate Change. 
    system_message="You are a creative writer. Please write a story based on Climate Change. Keep it at max 50 words. Make some grammatical errors and spelling issues.",
)

editor1 = AssistantAgent(
    name="Editor1",
    description="An editor agent that reviews and edits text.",
    model_client=model_client,
    # system_message="Edit the paragraph for Grammar.",
    system_message="Tell the grammatical mistakes  in point structure",

)

editor2= AssistantAgent(
    name="Editor2",
    description="An editor agent that reviews and edits text.",
    model_client=model_client,
    # system_message="Edit the paragraph for Style.
    system_message="Tell the spelling mistakes in point structure.",
)

reviewer = AssistantAgent(
    name="Reviewer",
    description="A reviewer agent that provides feedback on text.",
    model_client=model_client,
    system_message="Consolidate the Grammar and Style edits into a Final Version.",
)


In [28]:
from autogen_agentchat.teams import DiGraphBuilder,GraphFlow

builder = DiGraphBuilder()
builder.add_node(writer).add_node(editor1).add_node(editor2).add_node(reviewer)


In [29]:
# Fan out from Writer to Editors
builder.add_edge(writer, editor1)
builder.add_edge(writer, editor2)

# Fan in Both Editors into Reviewer
builder.add_edge(editor1, reviewer)
builder.add_edge(editor2, reviewer)

graph = builder.build()

In [10]:
graph

DiGraph(nodes={'Writer': DiGraphNode(name='Writer', edges=[DiGraphEdge(target='Editor1', condition=None), DiGraphEdge(target='Editor2', condition=None)], activation='all'), 'Editor1': DiGraphNode(name='Editor1', edges=[DiGraphEdge(target='Reviewer', condition=None)], activation='all'), 'Editor2': DiGraphNode(name='Editor2', edges=[DiGraphEdge(target='Reviewer', condition=None)], activation='all'), 'Reviewer': DiGraphNode(name='Reviewer', edges=[], activation='all')}, default_start_node=None)

In [13]:
builder.get_participants()

In [16]:
hex(id(writer)),hex(id(editor1)),hex(id(reviewer))

('0x10cd5cc20', '0x104379bd0', '0x10cd2f6f0')

In [30]:
flow = GraphFlow(participants=builder.get_participants(),graph=graph)



In [31]:
await Console(flow.run_stream(task='Write a short paragraph about Climate Change.'))

---------- TextMessage (user) ----------
Write a short paragraph about Climate Change.
---------- TextMessage (Writer) ----------
climmate changes making everything diferent, sudden storms and no snow in winter. Animals confused, trees loosing leafs in summer. People trying to undo or slow damages, planting trees and using less plastic. Hope lies in small actions everyday, maybe we can make the earth smile again. Time's running swiftly.
---------- TextMessage (Editor1) ----------
1. "climmate" should be "climate."
2. "diferent" should be "different."
3. "trees loosing leafs" should be "trees losing leaves."
4. "leafs" should be "leaves."
5. The sentence "climmate changes making everything diferent, sudden storms and no snow in winter." is incomplete. It should be revised for clarity.
6. Add a subject to the sentence: "Climmate changes making everything diferent, sudden storms and no snow in winter" could be corrected to "Climate change is making everything different, causing sudden sto

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='Write a short paragraph about Climate Change.', type='TextMessage'), TextMessage(source='Writer', models_usage=RequestUsage(prompt_tokens=51, completion_tokens=64), metadata={}, content="climmate changes making everything diferent, sudden storms and no snow in winter. Animals confused, trees loosing leafs in summer. People trying to undo or slow damages, planting trees and using less plastic. Hope lies in small actions everyday, maybe we can make the earth smile again. Time's running swiftly.", type='TextMessage'), TextMessage(source='Editor1', models_usage=RequestUsage(prompt_tokens=98, completion_tokens=265), metadata={}, content='1. "climmate" should be "climate."\n2. "diferent" should be "different."\n3. "trees loosing leafs" should be "trees losing leaves."\n4. "leafs" should be "leaves."\n5. The sentence "climmate changes making everything diferent, sudden storms and no snow in winter." is in